# Creating FABnet IPv4 Network

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet.   


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

import sys
sys.path.append("/home/brandon/fabric_additions/new_fabric_additions/jupyter-examples")

from plugins import Plugins
Plugins.load()

fablib = fablib_manager(output_type='HTML')

# fablib.output_type = 'HTML'

## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
#     print(f"{fablib.list_sites()}")
    fablib.get_resources().list()
except Exception as e:
    print(f"Exception: {e}")

Name,CPUs,Cores,RAM (G),Disk (G),Basic (100 Gbps NIC),ConnectX-6 (100 Gbps x2 NIC),ConnectX-5 (25 Gbps x2 NIC),P4510 (NVMe 1TB),Tesla T4 (GPU),RTX6000 (GPU)
MICH,6,190/192,1528/1536,60590/60600,380/381,2/2,2/2,10/10,2/2,3/3
UTAH,10,192/320,2048/2560,116360/116400,632/635,0/2,4/4,16/16,4/4,5/5
TACC,10,124/320,1896/2560,115330/116400,624/635,2/2,2/4,16/16,4/4,5/6
WASH,6,124/192,1264/1536,60560/60600,377/381,0/2,2/2,10/10,2/2,3/3
NCSA,6,190/192,1528/1536,60590/60600,380/381,2/2,2/2,10/10,2/2,3/3
DALL,6,192/192,1536/1536,60600/60600,381/381,2/2,2/2,10/10,2/2,3/3
MAX,10,288/320,2464/2560,116240/116400,609/635,2/2,4/4,16/16,4/4,6/6
MASS,4,124/128,1012/1024,55780/55800,252/254,2/2,0/0,6/6,0/0,3/3
SALT,6,92/192,1136/1536,60550/60600,377/381,0/2,2/2,10/10,2/2,3/3
STAR,12,270/384,2648/3072,121060/121200,756/762,0/2,6/6,20/20,6/6,6/6


## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [3]:
slice_name = 'MySlice'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

Sites: STAR, MASS


In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

Slice Name,MySlice
Slice ID,0296acd1-9813-49f2-9123-3bce94aefb4e
Slice State,StableOK
Lease End (UTC),2022-08-14 22:53:24 +0000



Retry: 11, Time: 136 sec


Name,ID,Site,Host,Cores,RAM,Disk,Image,Management IP,State,Error
Node1,f0ea20d3-e3e7-4ba2-ac3b-8497609d564b,STAR,star-w4.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3030:f816:3eff:feda:a408,Active,
Node2,08426e6c-5a35-4e14-8060-af2ebd9cb618,MASS,mass-w1.fabric-testbed.net,2,8,10,default_rocky_8,205.172.170.106,Active,



Time to stable 136 seconds
Running post_boot_config ... Time to post boot config 148 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical OS Interface,OS Interface
Node1-nic1-p1,Node1,net1,0,None,02:D2:B7:36:8C:AD,eth1,eth1
Node2-nic2-p1,Node2,net2,0,None,02:4B:B8:69:EF:DB,eth1,eth1



None

Time to print interfaces 157 seconds


## Observe the Slice's Attributes

### Print the slice

In [5]:
slice_name='MySlice'
try:
    slice = fablib.get_slice(name=slice_name)
#     print(f"{slice}")
    slice.show()
except Exception as e:
    print(f"Exception: {e}")

Slice Name,MySlice
Slice ID,0296acd1-9813-49f2-9123-3bce94aefb4e
Slice State,StableOK
Lease End (UTC),2022-08-14 22:53:24 +0000


## Print the Node List

In [6]:
try:
    slice = fablib.get_slice(name=slice_name)
#     print(f"{slice.list_nodes()}")
    slice.list_nodes()
except Exception as e:
    print(f"Exception: {e}")

Name,ID,Site,Host,Cores,RAM,Disk,Image,Management IP,State,Error
Node1,f0ea20d3-e3e7-4ba2-ac3b-8497609d564b,STAR,star-w4.fabric-testbed.net,2,8,10,default_rocky_8,2001:400:a100:3030:f816:3eff:feda:a408,Active,
Node2,08426e6c-5a35-4e14-8060-af2ebd9cb618,MASS,mass-w1.fabric-testbed.net,2,8,10,default_rocky_8,205.172.170.106,Active,


## Print the Node Details

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
#         print(f"{node}")
        node.show()
    slice.list_nodes(verbose=True)
except Exception as e:
    print(f"Exception: {e}")

ID,f0ea20d3-e3e7-4ba2-ac3b-8497609d564b
Name,Node1
Cores,2
RAM,8
Disk,10
Image,default_rocky_8
Image Type,qcow2
Host,star-w4.fabric-testbed.net
Site,STAR
Management IP,2001:400:a100:3030:f816:3eff:feda:a408
Reservation State,Active


ID,08426e6c-5a35-4e14-8060-af2ebd9cb618
Name,Node2
Cores,2
RAM,8
Disk,10
Image,default_rocky_8
Image Type,qcow2
Host,mass-w1.fabric-testbed.net
Site,MASS
Management IP,205.172.170.106
Reservation State,Active


Name,Site,ID,Cores,RAM,Disk,Image,SSH Command,Image Type,Host,Management IP,Reservation State,Error Message
Node1,STAR,f0ea20d3-e3e7-4ba2-ac3b-8497609d564b,2,8,10,default_rocky_8,ssh -i /home/brandon/fabric_additions/fabric_config/slice-key -J brice6_0051555767@bastion-1.fabric-testbed.net rocky@2001:400:a100:3030:f816:3eff:feda:a408,qcow2,star-w4.fabric-testbed.net,2001:400:a100:3030:f816:3eff:feda:a408,Active,
Node2,MASS,08426e6c-5a35-4e14-8060-af2ebd9cb618,2,8,10,default_rocky_8,ssh -i /home/brandon/fabric_additions/fabric_config/slice-key -J brice6_0051555767@bastion-1.fabric-testbed.net rocky@205.172.170.106,qcow2,mass-w1.fabric-testbed.net,205.172.170.106,Active,


## Print the Interfaces

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)
#     print(f"{slice.list_interfaces()}")
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

Name,Node,Network,Bandwidth,VLAN,MAC,Physical OS Interface,OS Interface
Node1-nic1-p1,Node1,net1,0,None,02:D2:B7:36:8C:AD,eth1,eth1
Node2-nic2-p1,Node2,net2,0,None,02:4B:B8:69:EF:DB,eth1,eth1


## Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Get the Assigned Subnet

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects. 

In [9]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    print(f"{network1}")
#     network1.show()
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    print(f"{network2}")
#     network2.show()
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------
ID                 aa740fd0-ac5e-4e6e-97ad-ac2ce3e2f53d
Name               net1
Layer              L3
Type               FABNetv4
Site               STAR
Gateway            10.129.129.1
L3 Subnet          10.129.129.0/24
Reservation State  Active
Error Message
-----------------  ------------------------------------
-----------------  ------------------------------------
ID                 88e2de65-c853-403c-8b36-223a92ff4d02
Name               net2
Layer              L3
Type               FABNetv4
Site               MASS
Gateway            10.131.129.1
L3 Subnet          10.131.129.0/24
Reservation State  Active
Error Message
-----------------  ------------------------------------


### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Then set a route from *this network* to the *other network* through the specified gateway.


Optionally, use the `node.execute()` method to show the results of adding the IP address and route.



In [10]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:d2:b7:36:8c:ad brd ff:ff:ff:ff:ff:ff
    inet 10.129.129.2/24 scope global eth1
       valid_lft forever preferred_lft forever

default via 10.20.4.1 dev eth0 proto dhcp metric 100 
10.20.4.0/23 dev eth0 proto kernel scope link src 10.20.4.199 metric 100 
10.129.129.0/24 dev eth1 proto kernel scope link src 10.129.129.2 
10.131.129.0/24 via 10.129.129.1 dev eth1 
169.254.169.254 via 10.20.4.11 dev eth0 proto dhcp metric 100 



### Configure Node2

Repeat the steps to add the next available IP to the second node and a route to the first network.

In [11]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name) 
    node2_addr = network2_available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:4b:b8:69:ef:db brd ff:ff:ff:ff:ff:ff
    inet 10.131.129.2/24 scope global eth1
       valid_lft forever preferred_lft forever

default via 10.20.4.1 dev eth0 proto dhcp metric 100 
10.20.4.0/23 dev eth0 proto kernel scope link src 10.20.5.93 metric 100 
10.129.129.0/24 via 10.131.129.1 dev eth1 
10.131.129.0/24 dev eth1 proto kernel scope link src 10.131.129.2 
169.254.169.254 via 10.20.4.1 dev eth0 proto dhcp metric 100 



## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [12]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 10.131.129.2 (10.131.129.2) 56(84) bytes of data.
64 bytes from 10.131.129.2: icmp_seq=1 ttl=61 time=154 ms
64 bytes from 10.131.129.2: icmp_seq=2 ttl=61 time=151 ms
64 bytes from 10.131.129.2: icmp_seq=3 ttl=61 time=151 ms
64 bytes from 10.131.129.2: icmp_seq=4 ttl=61 time=151 ms
64 bytes from 10.131.129.2: icmp_seq=5 ttl=61 time=151 ms

--- 10.131.129.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 150.777/151.366/153.694/1.189 ms




## Delete the Slice

Please delete your slice when you are done with your experiment.

In [13]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")